In [ ]:
# Data clustering

import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
import matplotlib.pyplot as plt




class Particle:
    def __init__(self, n_clusters, data,use_kmeans=True,c1=2.05, c2=2.05): # constructor
        
        col_max=list(data.max(axis=0))
        col_min=list(data.min(axis=0))
        bounds=np.column_stack([col_max,col_min])
        self.ocentroids_pos=np.zeros((3,4))                                  #cluster, feature
        self.centroids_pos=np.zeros((3,4))                                   #cluster, feature
        self.n_clusters = n_clusters
        if use_kmeans == True:
            k_means = KMeans(n_clusters=self.n_clusters)
            k_means.fit(data)
            self.centroids_pos = k_means.cluster_centers_
                 
        else:
            samples = np.zeros((3,4))                                       # cluster, feature
            for i in range(3):                                              #no of cluster
                R=np.random.uniform(0,1)
                for j in range(4):                                           # no of feature
                    R=np.random.uniform(0,1)
                    s1 = bounds[j,1]+R*(bounds[j,0]-bounds[j,1])
                    samples[i,j]=s1
                    self.centroids_pos=samples
        
        
            
            osamples=np.zeros((3,4))                                       #cluster, feature
            for i in range(3):                                             #Q-opposition based learning method  # no of cluster
                for j in range(4):                                          #no of features
                    M=((bounds[j,0]+bounds[j,1])/2)
                    N=M*2
                    if(self.centroids_pos[i,j]<M):
                        s2=M+((N-self.centroids_pos[i,j])-M)*np.random.uniform(0,1)
                        osamples[i,j]=s2
                        self.ocentroids_pos=osamples
                    else:
                        a=N-self.centroids_pos[i,j]
                        b=N-self.centroids_pos[i,j]
                        s2=a+(M-b)*np.random.uniform(0,1)                               
                        osamples[i,j]=s2
                        self.ocentroids_pos=osamples
                                   
        
        self.pb_val = np.inf
        self.pb_val = np.inf
        self.pb_pos = self.centroids_pos.copy() 
        self.velocity = np.zeros_like(self.centroids_pos) 
        # best data clustering so far
        self.pb_clustering = None
        self.c1 = c1
        self.c2 = c2
        self.fk = 0.01
            
    def update_pb(self, data: np.ndarray):                       
         distances = self._get_distances(data=data)
         clusters = np.argmin(distances, axis=0)  
         clusters_ids = np.unique(clusters)   
         new_val=self.withinclustersumofsquarefitness_function(clusters=clusters, distances=distances)
         if new_val < self.pb_val:
            self.pb_val = new_val
            self.pb_pos = self.centroids_pos.copy()
            self.pb_clustering = clusters.copy()
                  
    
    def nupdate_pb(self, data: np.ndarray):                       
         distances = self._get_distances(data=data)
         clusters = np.argmin(distances, axis=0)  
         clusters_ids = np.unique(clusters)
         new_val=self.withinclustersumofsquarefitness_function(clusters=clusters, distances=distances)
         return(new_val)  
    
    
    
    
    
    def oupdate_pb(self, data: np.ndarray):                      
         distances = self._oget_distances(data=data)
         clusters = np.argmin(distances, axis=0)  
         clusters_ids = np.unique(clusters)
         new_val=self.withinclustersumofsquarefitness_function(clusters=clusters, distances=distances)
         return(new_val)
         
                                 
            
            
    def update_velocity(self, df, max_itr, cur_itr, gb_pos: np.ndarray):
        """
        Updates new velocity based on the current velocity, personal best position so far, and the swarm (global) best
        position so far.
        :param gb_pos: vector of best centroid positions among all particles so far
        :return:
        """
        print()
        self.w=0.9
        self.w=0.9-(((0.9-0.2)/max_itr)*cur_itr)
        #self.w = 0.2 + (((0.9 - 0.2) / max_itr) * cur_itr)
        self.fk=4*self.fk*(1-self.fk)
        self.w=self.w*self.fk
        self.velocity = self.w * self.velocity + \
                        self.c1 * np.random.random() * (self.pb_pos - self.centroids_pos) + \
                        self.c2 * np.random.random() * (gb_pos - self.centroids_pos)
        phi=self.c1+self.c2
        cfk=2/abs(2-phi-math.sqrt(math.pow(phi,2)-4*phi))
        self.velocity=cfk*self.velocity
        col_max=list(df.max(axis=0))
        col_min=list(df.min(axis=0))
        bounds=np.column_stack([col_max,col_min])
        #print("bounds are:\n",bounds)
        col_index=self.velocity.shape[1]
        l_max=[]
        l_min=[]
        for i in range(col_index):
            b=bounds[i,:]
            v_max=(b[0]-b[1])/15                                                 # velocity range
            v_min=-v_max
            l_max.append(v_max)
            #print("v max:", v_max)
            #print("v min:",v_min)
            l_min=[-i for i in l_max]
        bounds=np.column_stack([l_min,l_max])
        col_index=self.velocity.shape[1]
        for i in range(col_index):
            b=bounds[i,:]
            a1=self.velocity[:,i]<b[0]
            for j in range(a1.shape[0]):
                if a1[j]:
                    self.velocity[j,i]=b[0]
            a2=self.velocity[:,i]>b[1]
            for k in range(a2.shape[0]):
                if a2[k]:
                    self.velocity[k,i]=b[1]
        
             
    def move_centroids(self, gb_pos,df: np.ndarray, max_itr, cur_itr):  
              
        self.update_velocity(df,max_itr, cur_itr,gb_pos=gb_pos)  #made changes
        new_pos = self.centroids_pos + self.velocity 
        col_max=list(df.max(axis=0))
        col_min=list(df.min(axis=0))
        bounds=np.column_stack([col_min,col_max])
        col_index=new_pos.shape[1]
        #print("new pos shape 1:", col_index)
        for i in range(col_index):
            b=bounds[i,:]
            a1=new_pos[:,i]<b[0]
            for j in range(a1.shape[0]):
                if a1[j]:
                    new_pos[j,i]=b[0]
            a2=new_pos[:,i]>b[1]
            for k in range(a2.shape[0]):
                if a2[k]:
                    new_pos[k,i]=b[1]
        
        
        self.centroids_pos = new_pos.copy()
        
                    
            
            
    def _get_distances(self, data: np.ndarray) -> np.ndarray:
        
        distances = []
        for centroid in self.centroids_pos:
            d = np.linalg.norm(data-centroid, axis=1)  
            distances.append(d)
        distances = np.array(distances)
        return distances
    
    def _oget_distances(self, data: np.ndarray) -> np.ndarray:
        
        
        distances = []
        
        for centroid in self.ocentroids_pos:
            
            d = np.linalg.norm(data-centroid, axis=1)  
            distances.append(d)
        distances = np.array(distances)
        return distances
    
    
    def _fitness_function(self, clusters: np.ndarray, distances: np.ndarray) -> float:
        
        J = 0.0
        for i in range(self.n_clusters):
            p = np.where(clusters == i)[0]  
            if len(p):
                d = sum(distances[i][p])
                #print("d is",d)
                d /= len(p)
                J += d
        J /= self.n_clusters
        print("Quantization value:",J)
        return J
        
    def withinclustersumofsquarefitness_function(self, clusters: np.ndarray, distances: np.ndarray) -> float:
        
        J = 0.0
        for i in range(self.n_clusters):
            p = np.where(clusters == i)[0] 
            if len(p):
                #d = sum(distances[i][p]**2)
                d = sum(distances[i][p])
                J += d
                
        return J
    

    def davis(self,data:np.ndarray,clusters)-> float:
        davis=davies_bouldin_score(data_points, clusters)
        return davis
        
    

In [ ]:
from typing import Tuple
import math
import numpy as np
import matplotlib.pyplot as plt

class PSOClusteringSwarm:
    def __init__(self, n_clusters: int, n_particles: int, kcentroids:np.ndarray,data: np.ndarray, hybrid=True,c1=2.05, c2=2.05): 
        
        self.n_clusters = n_clusters
        self.n_particles = n_particles
        self.data = data
        self.normal_fitness=[]

        self.particles = []
        self.newparticles=[]
        self.newparticleso=[]
        self.combineparticles=[] 
        self.gb_pos = None  
        self.gb_val = np.inf 
        self.gb_clustering = None
        self.gb_each_itr=[]
        self._generate_particles(hybrid,c1, c2) #method call
        
    
    
    def _print_initial(self, iteration, plot):
        print('*** Initialing swarm with', self.n_particles, 'PARTICLES, ', self.n_clusters, 'CLUSTERS with', iteration,
              'MAX ITERATIONS and with PLOT =', plot, '***')
        print('Data=', self.data.shape[0], 'points in', self.data.shape[1], 'dimensions')

       

    def _generate_particles(self, hybrid: bool, c1: float, c2: float):  # hybrid is true
        """
        Generates particles with k clusters and t-dimensional points
        :return:
        """
        for i in range(self.n_particles):
            if i == 0 and hybrid:                                                 #
                particle = Particle(n_clusters=self.n_clusters, data=self.data, use_kmeans=hybrid, c1=c1, c2=c2)# con. call
                self.particles.append(particle)
            
            else:
                particle = Particle(n_clusters=self.n_clusters, data=self.data, use_kmeans=False, c1=c1, c2=c2)
                self.particles.append(particle)
            
        for particle in self.particles:
            v1=particle.nupdate_pb(data=self.data)
            self.normal_fitness.append(v1)
            v2=particle.oupdate_pb(data=self.data)                
            self.normal_fitness.append(v2) 
            print()
        self.sorted_fitness=sorted(self.normal_fitness)                                   # sorted fitness
        
    
        for j in range(len(self.particles)):
            self.newparticles.append(self.particles[j].centroids_pos)
            self.newparticleso.append(self.particles[j].ocentroids_pos)
                


        ###############################################################################################
        
        for i in range(0,30): 
            
                self.combineparticles.append(self.particles[i].centroids_pos)
            
                self.combineparticles.append(self.particles[i].ocentroids_pos)
                
        
        for particle in self.particles:
            for i in range(0,30):   
                fitness=self.sorted_fitness[i]
                for j in range(0,60):    
                    if self.normal_fitness[j]==fitness:
                        self.particles[i].centroids_pos=self.combineparticles[j]
        
        
     ####################################################################################################################   
        
        
                    
                    
        for k in range(self.n_particles):
            print()
                 
    
    
    def update_gb(self, particle):
        if particle.pb_val < self.gb_val:
            self.gb_val = particle.pb_val
            self.gb_pos = particle.pb_pos.copy()
            self.gb_clustering = particle.pb_clustering.copy()
        
            
            
    
    
    def start(self, iteration,data: np.ndarray,plot=False) -> Tuple[np.ndarray, float]:                         
        z=1
        print("iteration value", iteration)
        if (z==1):
                for particle in self.particles:
                    #kmeans = KMeans(n_clusters=3, init=self.particles[1].centroids_pos)
                    #kmeans.fit(data)
                    #self.particles[1].centroids_pos = k_means.cluster_centers_
                    #print("self.particles[1].centroids_pos:",self.particles[1].centroids_pos)
                   
                    #kmeans = KMeans(n_clusters=10, init=self.particles[2].centroids_pos)
                    #kmeans.fit(data)
                    #self.particles[2].centroids_pos = k_means.cluster_centers_
                    
                    #kmeans = KMeans(n_clusters=10, init=self.particles[3].centroids_pos)
                    #kmeans.fit(data)
                    #self.particles[3].centroids_pos = k_means.cluster_centers_
                    

                    
                    z=z+1
        
                    
        self._print_initial(iteration, plot)
        progress = []
        gb_val1=0
        temp=0
        diff=math.inf
        for i in range(iteration):
            if i % 200 == 0:
                clusters = self.gb_clustering   # none assign to clusters
                              
            for particle in self.particles:
                particle.update_pb(data=self.data)
                self.update_gb(particle=particle)
                
                                        
            
            for particle in self.particles:
                particle.move_centroids(gb_pos=self.gb_pos, df=data, max_itr=iteration, cur_itr=i+1) 
                diff = np.abs(gb_val1 - self.gb_val)
            gb_val1=self.gb_val
            
            progress.append([self.gb_pos, self.gb_clustering, self.gb_val])
            self.gb_each_itr.append(self.gb_val)
        print('Finished!')
        return self.gb_clustering, self.gb_val,self.gb_pos
    
    

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score


plot=True
data_points = pd.read_csv('iris.txt',sep=',',header=None)
data_points = data_points[[0,1,2,3]]
#data_points=data_points.dropna(subset=[6])                    #for cancer data set missing value
#data_points=data_points.dropna(subset=[33])                    # for dermatology           
data_points = data_points.values
#k_means = KMeans(n_clusters=3)
#k_means.fit(data_points)
#kcentroids = k_means.cluster_centers_

pso = PSOClusteringSwarm(n_clusters=3, n_particles=30, kcentroids = k_means.cluster_centers_,data=data_points,hybrid=False) 
gb_clustering, gb_val,gb_pos=pso.start(iteration=200, data=data_points,plot=plot)   
#print("\nGlobal best clustering",gb_clustering)
print("\nGlobal best value:", gb_val)
print("\nGlobal best position", gb_pos)
silhouette_avg = silhouette_score(data_points, gb_clustering)
print("Sillouete score:",silhouette_avg)
davies_bouldin=davies_bouldin_score(data_points, gb_clustering)
print("davis score:",davies_bouldin)
print("\nGlobal best position", gb_clustering)